In [0]:
# pyspark functions
from pyspark.sql.functions import *
# URL processing
import urllib

In [0]:
# Define the path to the Delta table
delta_table_path = "dbfs:/user/hive/warehouse/authentication_credentials"

# Read the Delta table to a Spark DataFrame
aws_keys_df = spark.read.format("delta").load(delta_table_path)

In [0]:
# Get the AWS access key and secret key from the spark dataframe
ACCESS_KEY = aws_keys_df.select('Access key ID').collect()[0]['Access key ID']
SECRET_KEY = aws_keys_df.select('Secret access key').collect()[0]['Secret access key']
# Encode the secrete key
ENCODED_SECRET_KEY = urllib.parse.quote(string=SECRET_KEY, safe="")

In [0]:
# AWS S3 bucket name
AWS_S3_BUCKET = "user-0affc011d3cf-bucket"
# Mount name for the bucket
MOUNT_NAME = "/mnt/0affc011d3cf-mount"
# Source url
SOURCE_URL = "s3n://{0}:{1}@{2}".format(ACCESS_KEY, ENCODED_SECRET_KEY, AWS_S3_BUCKET)
# Mount the drive
dbutils.fs.mount(SOURCE_URL, MOUNT_NAME)


path,name,size,modificationTime
dbfs:/FileStore/,FileStore/,0,1721931156862
dbfs:/Users/,Users/,0,1721931156862
dbfs:/Volume/,Volume/,0,0
dbfs:/Volumes/,Volumes/,0,0
dbfs:/_delta_log/,_delta_log/,0,1721931156862
dbfs:/checkpoint/,checkpoint/,0,1721931156862
dbfs:/custom/,custom/,0,1721931156862
dbfs:/databricks-datasets/,databricks-datasets/,0,0
dbfs:/databricks-results/,databricks-results/,0,0
dbfs:/dbfs/,dbfs/,0,1721931156862


In [0]:
# List the contents of the mount point
dbutils.fs.ls('/mnt/0affc011d3cf-mount')


Out[5]: [FileInfo(path='dbfs:/mnt/0affc011d3cf-mount/kafka-connect-s3/', name='kafka-connect-s3/', size=0, modificationTime=1721931431432),
 FileInfo(path='dbfs:/mnt/0affc011d3cf-mount/topics/', name='topics/', size=0, modificationTime=1721931431432)]

In [0]:
# Check topics 

display(dbutils.fs.ls("/mnt/0affc011d3cf-mount/topics"))

path,name,size,modificationTime
dbfs:/mnt/0affc011d3cf-mount/topics/0affc011d3cf.geo/,0affc011d3cf.geo/,0,1721933488225
dbfs:/mnt/0affc011d3cf-mount/topics/0affc011d3cf.pin/,0affc011d3cf.pin/,0,1721933488225
dbfs:/mnt/0affc011d3cf-mount/topics/0affc011d3cf.user/,0affc011d3cf.user/,0,1721933488225


In [0]:
%sql
-- Disable format checks during the reading of Delta tables
SET spark.databricks.delta.formatCheck.enabled=false

In [0]:
# File location and type
# Asterisk(*) indicates reading all the content of the specified file that have .json extension
pin_location = "/mnt/0affc011d3cf-mount/topics/0affc011d3cf.pin/partition=0/*.json" 
geo_location = "/mnt/0affc011d3cf-mount/topics/0affc011d3cf.geo/partition=0/*.json" 
user_location = "/mnt/0affc011d3cf-mount/topics/0affc011d3cf.user/partition=0/*.json" 

file_type = "json"

# Ask Spark to infer the schema
infer_schema = "true"

# Read in JSONs from mounted S3 bucket
# load pin data
df_pin = spark.read.format(file_type).option("inferSchema", infer_schema).load(pin_location)

# load geo data
df_geo = spark.read.format(file_type).option("inferSchema", infer_schema).load(geo_location)

# load user data
df_user = spark.read.format(file_type).option("inferSchema", infer_schema).load(user_location)

# Display Spark dataframe to check its content
df_pin.limit(10).display()
df_geo.limit(10).display()
df_user.limit(10).display()

category,description,downloaded,follower_count,image_src,index,is_image_or_video,poster_name,save_location,tag_list,title,unique_id
mens-fashion,スポーツミックススタイルやアスレジャースタイルの台頭によって、すっかりおなじみとなった「ジョガーパンツ」。スニーカーと相性抜群なアイテムであり、フーディを合わせたカジュアルなスタイルからジャケットを羽織ったドレスライクなスタイルまで幅広くフィットする。今回はそんな「ジョガーパンツ」にフォーカスして注目の着こなし&アイテムを紹介！,1,122k,https://i.pinimg.com/originals/5d/31/e4/5d31e49fada653798f7c8f4c47f65d14.jpg,7491,image,OTOKOMAE/男前研究所,Local save in /data/mens-fashion,"Streetwear,Mens Casual Hats,Fashion Week Hommes,Herren Style,Moda Blog,Look Man,La Mode Masculine,Outfits With Converse,Black Converse",ジョガーパンツでメンズコーデの足元を軽快にこなす！ | メンズファッションメディア OTOKOMAE,0bfd8ee2-8bc8-4d43-8a43-3f9e5f9678bf
event-planning,もうすぐ親友の誕生日。何年も共に過ごしてきただけに、毎年の誕生祝いでほとんどのアイデアは出し尽くしてしまい悩み中、、、という方に。今年はちょっと変わったバースデイサプライズで、大好きな親友の素敵な笑顔を引き出してみませんか。,1,256k,https://i.pinimg.com/originals/c8/d7/e6/c8d7e693a171205650a1ce50036c45d4.jpg,4362,image,LOCARI,Local save in /data/event-planning,"Love Ballons,Balloons,Wedding Proposals,Marriage Proposals,Wedding Tags,Wedding Gifts,Wedding Ideas,Diy Wedding,Proposition Romantique",誕生日に親友を必ず笑顔にする11のサプライズ・アイデア♡,3d2ca512-9a37-4bb5-93bc-fca20aaafdce
vehicles,"Пинтерест точно лучше инстаграмма #traffic #траффикнашевсе #траффик_наше_все Траффик наше все! Продвижение, SEO, SMM 👍🏻Я влюблен в эту программ",1,283,https://i.pinimg.com/originals/84/a3/17/84a317acbdd5fca1c3b80ea30d45c370.jpg,10981,image,Edward Kramli,Local save in /data/vehicles,"Luxury Sports Cars,Top Luxury Cars,Sport Cars,Exotic Sports Cars,Mercedes Benz Amg,Carros Mercedes Benz,Benz Car,Carros Lamborghini,Carros Audi","Пинтерест точно лучше инстаграмма.. | Траффик наше все! Продвижение, SEO, SMM",682f6f97-33d6-4b5b-bae7-b94615ea3091
christmas,"6.5Ft + Pearl & Pine Cone Design - This Pearl & Pine Cone string light as a Christmas decor light. 6.5 Foot/2 Meters plated copper wire and 20 LED Lights, The led string lights…",1,5k,https://i.pinimg.com/originals/3f/94/40/3f94400a8d9878d76db9d68126161b13.jpg,2244,image,Wear24-7,Local save in /data/christmas,"Mini Christmas Tree Decorations,Christmas Pine Cones,Christmas Fairy Lights,Led Fairy Lights,Xmas Lights,Xmas Wreaths,Christmas Bells,Outdoor Christmas,Christmas Holidays",Christmas String Lights with Pine Cone Red Pearl Bell Garland with Lights 20 LED Warm White Battery Operated Christmas Tree Decor Light for Christmas Holiday Indoor Table Party Decoration,20485f66-1504-4272-bb96-049d7564abdd
christmas,"Christmas decorating ideas for porches. Beautiful holiday decor ideas for front porches both small and large. Outdoor decorations like sleds, lanterns, Christmas trees, wreaths,…",1,46k,https://i.pinimg.com/videos/thumbnails/originals/40/83/f5/4083f5b4971bf235f89a4784ab87271e.0000001.jpg,2482,video,"Life on Summerhill | Home, Holiday Decor & DIY Website",Local save in /data/christmas,"Diy Christmas Decorations For Home,Farmhouse Christmas Decor,Christmas Home,Christmas Holidays,Christmas Front Porches,How To Decorate For Christmas,Christmas Porch Ideas,Christmas Decorating Ideas,Large Outdoor Christmas Decorations",FORNT PORCH CHRISTMAS DECORATING IDEAS,08604f20-fa17-4b9a-9949-781717eca6cd
christmas,Toddler Christmas activity using sticky contact paper. Great for developing fine motor skills and more. #kbn #ohchristmastree #stickycontactpaper #handmadetree #kidschristmas #c…,1,99k,https://i.pinimg.com/originals/a6/61/c8/a661c87eee093cf82d3697b05b9aaee3.jpg,2205,image,Laughing Kids Learn | Play Ideas for Kids | Crafts + Activities for Teachers + Parents,Local save in /data/christmas,"Christmas Activities For Toddlers,Preschool Christmas,Holiday Activities,Christmas Paper,Craft Activities,Preschool Crafts,Christmas Themes,Christmas Decorations,Toddler Christmas Crafts",Toddler Christmas activity using sticky contact paper - Laughing Kids Learn,24cbe878-d95a-4902-baef-f831fcb9bbd1
tattoos,beautiful Dragon ❌ flowers hip tattoo ✿ Strelkov Mikhail ®️ ✧ @strelkovtattoo ✧ Moscow 🇷🇺 ✿ 💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖🐉ꕥ💖 #hiptattoo #floraltattoo #awesomeness #girltatto

country,ind,latitude,longitude,timestamp
British Indian Ocean Territory (Chagos Archipelago),8221,-20.5574,-54.4834,2021-12-29T06:33:46
British Indian Ocean Territory (Chagos Archipelago),7151,-14.6744,-75.3714,2020-06-05T23:37:24
British Indian Ocean Territory (Chagos Archipelago),7569,-86.5675,-149.565,2018-10-16T08:40:26
British Indian Ocean Territory (Chagos Archipelago),6731,-86.5675,-149.565,2021-06-08T19:10:49
British Indian Ocean Territory (Chagos Archipelago),9345,22.1505,-158.823,2020-06-20T08:22:49
Antarctica (the territory South of 60 deg S),9127,-77.9931,-175.682,2021-09-25T14:31:22
Antarctica (the territory South of 60 deg S),5855,-84.9073,-105.769,2022-04-19T19:50:22
Antarctica (the territory South of 60 deg S),7743,-24.9999,-92.9126,2020-06-02T06:18:41
Antarctica (the territory South of 60 deg S),2294,-88.4642,-171.061,2020-05-04T09:26:02
South Georgia and the South Sandwich Islands,10024,-24.7257,89.1335,2021-03-19T02:41:59


age,date_joined,first_name,ind,last_name
27,2017-06-10T17:31:24,Jacqueline,9840,Henderson
48,2016-02-27T16:57:44,Christopher,1768,Hamilton
27,2016-03-08T13:38:37,Christopher,6189,Bradshaw
24,2016-04-06T20:17:29,Christopher,8221,Edwards
30,2016-03-02T23:44:12,Christopher,813,Gonzalez
49,2016-05-23T19:42:19,Jennifer,10190,Blanchard
23,2015-10-31T19:20:09,Alexandria,6051,Anderson
20,2015-10-23T04:13:23,Alexandria,3929,Alvarado
55,2016-12-09T16:02:06,Courtney,9389,Richardson
20,2015-10-23T04:13:23,Alexandria,4272,Alvarado
